In [ ]:
python -m spacy download en_core_web_sm

In [1]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

import pandas as pd
import numpy as np
from scipy.stats import randint

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

## algorithm
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()


df = pd.read_csv('stock_data.csv')
df['Text'] = df['Text'].astype(str)
df['Date'] = pd.to_datetime(df['Date'])
df['Sentiment'] = df['Sentiment'].astype(np.int64)
print(df.info())

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

to_remove = ['up', 'down','low','high','below','over','biggest','big','less','fall']
new_stopwords = set(stopwords.words('english')).difference(to_remove)
stemmer = SnowballStemmer(language='english')
def text_data_cleaning(sentence):
    sent = preprocess_text(sentence)
    doc = nltk.word_tokenize(sent)
    lemma =[lemmatizer.lemmatize(word, pos ="v") for word in doc]
    return append_message(lemma)
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''
    sentence = sen.lower()
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = re.sub('rs|cr|crore|point|points|pt', ' ', sentence)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence) 
    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  
    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(new_stopwords) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

def append_message(text):
  str = " "
  return (str.join(text))

df['Tok_text'] = df['Text'].apply(text_data_cleaning)
df['Tok_text'] = df['Tok_text'].astype(str)
tf_idf_vect = TfidfVectorizer()

df2 = df.iloc[:,2:]

df_train, df_test = train_test_split(df2, test_size=0.3, random_state=42,
                                     stratify=df2['Sentiment'])
X_train = tf_idf_vect.fit_transform(df_train['Tok_text'])
y_train = df_train['Sentiment']
X_test = tf_idf_vect.transform(df_test['Tok_text'])
y_test = df_test['Sentiment']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32658 entries, 0 to 32657
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       32658 non-null  object        
 1   Date       32658 non-null  datetime64[ns]
 2   Sentiment  32658 non-null  int64         
 3   Tok_text   32656 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1020.7+ KB
None


In [6]:
pd.set_option('display.max_column', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [33]:
# clf1 = SGDClassifier(alpha=0.0001,loss='modified_huber')
# clf1.fit(X_train,y_train)
# y_pred = clf1.predict(X_test)
# print(classification_report(y_test, y_pred))
# score = accuracy_score(y_pred,y_test)
# print(score,"SDG"

# clf2 = LogisticRegression(C=10,multi_class='multinomial',penalty='l2',solver='lbfgs')
# clf2.fit(X_train,y_train)
# y_pred2 = clf2.predict(X_test)
# print(classification_report(y_test,y_pred2))
# score1 = accuracy_score(y_pred2,y_test)
# print(score, "LR")

              precision    recall  f1-score   support

           0       0.82      0.75      0.78      2062
           1       0.88      0.93      0.91      5929
           2       0.84      0.77      0.80      1807

    accuracy                           0.86      9798
   macro avg       0.85      0.82      0.83      9798
weighted avg       0.86      0.86      0.86      9798

0.8641559501939171 SDG
              precision    recall  f1-score   support

           0       0.81      0.75      0.78      2062
           1       0.89      0.92      0.90      5929
           2       0.82      0.77      0.79      1807

    accuracy                           0.86      9798
   macro avg       0.84      0.82      0.83      9798
weighted avg       0.86      0.86      0.86      9798

0.8641559501939171 LR


c:\Users\kings\OneDrive\Documents\Dev\Stock-Analysis\envr\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [40]:
rf_model = RandomForestClassifier()

# define the hyperparameter search space
param_dist = {
    'n_estimators': randint(50, 250),
    'max_depth': [None] + list(randint(3, 20).rvs(size=5)),
    'max_features': randint(1, X_train.shape[1]),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'n_jobs':[-1]
}

# define the randomized search object with cross-validation
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)

# fit the randomized search object to the training data
random_search.fit(X_train, y_train)

# print the best hyperparameters
print("Best hyperparameters:", random_search.best_params_)
best_model = random_search.best_estimator_

# make predictions on the test data
y_pred = best_model.predict(X_test)

# evaluate the model
print(classification_report(y_test, y_pred))

c:\Users\kings\OneDrive\Documents\Dev\Stock-Analysis\envr\lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


              precision    recall  f1-score   support

           0       0.81      0.63      0.71      2062
           1       0.84      0.94      0.88      5929
           2       0.83      0.71      0.76      1807

    accuracy                           0.83      9798
   macro avg       0.83      0.76      0.79      9798
weighted avg       0.83      0.83      0.82      9798

0.8304756072667892 SDG


In [42]:
from sklearn.model_selection import GridSearchCV
lr_model = LogisticRegression(multi_class='ovr')

# define the hyperparameter search space
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'max_iter': [100, 200, 500],
    'n_jobs':[-1]
}

# define the grid search object with cross-validation
grid_search = GridSearchCV(estimator=lr_model, param_grid=param_grid, cv=5, scoring='accuracy')

# fit the grid search object to the training data
grid_search.fit(X_train, y_train)

# print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# get the best model
best_model = grid_search.best_estimator_

# make predictions on the test data
y_pred = best_model.predict(X_test)

# evaluate the model
print(classification_report(y_test, y_pred))

c:\Users\kings\OneDrive\Documents\Dev\Stock-Analysis\envr\lib\site-packages\sklearn\linear_model\_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
c:\Users\kings\OneDrive\Documents\Dev\Stock-Analysis\envr\lib\site-packages\sklearn\linear_model\_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
c:\Users\kings\OneDrive\Documents\Dev\Stock-Analysis\envr\lib\site-packages\sklearn\linear_model\_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
c:\Users\kings\OneDrive\Documents\Dev\Stock-Analysis\envr\lib\site-packages\sklearn\linear_model\_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
c:\Users\kings\OneDrive\Documents\Dev\Stock-

Best hyperparameters: {'C': 10, 'max_iter': 100, 'n_jobs': -1, 'penalty': 'l2', 'solver': 'lbfgs'}
              precision    recall  f1-score   support

           0       0.82      0.75      0.78      2062
           1       0.89      0.93      0.91      5929
           2       0.83      0.77      0.80      1807

    accuracy                           0.86      9798
   macro avg       0.84      0.82      0.83      9798
weighted avg       0.86      0.86      0.86      9798



In [ ]:
from sklearn.svm import SVC
svm_model = SVC(decision_function_shape='ovr')

# define the hyperparameter search space
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'],
}

# define the grid search object with cross-validation
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

# print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# get the best model
best_model = grid_search.best_estimator_

# make predictions on the test data
y_pred = best_model.predict(X_test)

# evaluate the model
print(classification_report(y_test, y_pred))

In [43]:
from sklearn.ensemble import GradientBoostingClassifier
gb_model = GradientBoostingClassifier()

# define the hyperparameter search space
param_grid = {
    'learning_rate': [0.05, 0.1, 0.25, 0.5, 0.75, 1.0],
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [3, 4, 5, 6, 7],
    'min_samples_split': [2, 4, 6, 8],
    'min_samples_leaf': [1, 2, 4, 6],
}

# define the grid search object with cross-validation
grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

# print the best hyperparameters
print("Best hyperparameters:", grid_search.best_params_)

# get the best model
best_model = grid_search.best_estimator_

# make predictions on the test data
y_pred = best_model.predict(X_test)

# evaluate the model
print(classification_report(y_test, y_pred))

KeyboardInterrupt: 

In [ ]:
# define the SVM classifier
svm_model = SVC(probability=True)

# define the hyperparameter search space
param_dist = {
    'C': expon(scale=1),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto'] + list(expon(scale=1).rvs(size=5)),
    'degree': range(1, 6),
    'coef0': expon(scale=1)
}

# define the randomized search object with cross-validation
random_search = RandomizedSearchCV(estimator=svm_model, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)

# print the best hyperparameters
print("Best hyperparameters:", random_search.best_params_)

# get the best model
best_model = random_search.best_estimator_

# make predictions on the test data
y_pred = best_model.predict(X_test)

# evaluate the model
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier()

# define the hyperparameter search space
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50), (100,100)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [100, 200, 300],
}

# define the grid search object with cross-validation
grid_search = GridSearchCV(estimator=mlp_model, param_grid=param_grid, cv=5, scoring='accuracy')

# fit the grid search object to the training data
grid_search.fit(X_train, y_train)
print("Best hyperparameters:", grid_search.best_params_)

# get the best model
best_model = grid_search.best_estimator_

# make predictions on the test data
y_pred = best_model.predict(X_test)

# evaluate the model
print(classification_report(y_test, y_pred))

In [ ]:
sgd_model = SGDClassifier(loss='log', random_state=42)

# define the hyperparameter search space
param_grid = {
    'alpha': [0.0001, 0.001, 0.01],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'max_iter': [100, 200, 300],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
}

# define the grid search object with cross-validation
grid_search = GridSearchCV(estimator=sgd_model, param_grid=param_grid, cv=5, scoring='accuracy')

# fit the grid search object to the training data
grid_search.fit(X_train, y_train)

print("Best hyperparameters:", grid_search.best_params_)

# get the best model
best_model = grid_search.best_estimator_

# make predictions on the test data
y_pred = best_model.predict(X_test)

# evaluate the model
print(classification_report(y_test, y_pred))

In [6]:
y_pred = clf1.predict(x_test)
check = pd.DataFrame({'Text':test_data['Text'],'Date':test_data['Date'],'Sentiment':y_pred})
check.to_csv('check.csv')

In [11]:
df1 = pd.read_csv('check.csv')
df1.drop(columns=['Unnamed: 0'], axis=1,  inplace=True)
df1.rename(columns={'-1' : 'Sentiment'},inplace=True)
print(df1.info())
print(df.info())
# df.drop(columns=['Tok_text'], axis=1,  inplace=True)
fg = df.append(df1,ignore_index=True)
fg.info()
fg.to_csv('stock_data.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4698 entries, 0 to 4697
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       4698 non-null   object
 1   Date       4698 non-null   object
 2   Sentiment  4698 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 110.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27008 entries, 0 to 27007
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       27008 non-null  object        
 1   Date       27008 non-null  datetime64[ns]
 2   Sentiment  27008 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 633.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31706 entries, 0 to 31705
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       31706 non-null  objec

C:\Users\kings\AppData\Local\Temp\ipykernel_21816\1118238801.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fg = df.append(df1,ignore_index=True)


In [288]:
import pandas as pd
import numpy as np
df = pd.read_csv('stock_data.csv')
df['Text'] = df['Text'].astype(str)
df.drop_duplicates(subset="Text",
                     keep=False, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df['Sentiment'] = df['Sentiment'].astype(np.int64)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 32658 entries, 0 to 32657
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       32658 non-null  object        
 1   Date       32658 non-null  datetime64[ns]
 2   Sentiment  32658 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1020.6+ KB
None


In [13]:
df['Sentiment'].value_counts()

1    19760
0     6874
2     6024
Name: Sentiment, dtype: int64

In [286]:
# df = df[df['Text'].str.contains('Wall St')==False]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32658 entries, 0 to 32657
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Text       32658 non-null  object        
 1   Date       32658 non-null  datetime64[ns]
 2   Sentiment  32658 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1020.6+ KB


In [14]:
df.to_csv('stock_data.csv',index=False)

In [265]:
df = df[df['Text'].str.contains(' Wall St ')!=True]

In [12]:
df.loc[df['Text'].str.contains(' dividend') & (df['Text'].str.contains('jump|rise|gain') ==False)  & (df['Sentiment']==1), 'Sentiment'] = 0

In [ ]:
27630	Dow closes below 20,000	1
28650	Nifty50 below 18,150	1
28804	stock lists 7% below issue price	1
29325	crude below $100 per barrel	1
30225	Sensex below 61k	1
31394	Nifty ends below 14,700	1
31487	Sensex tanks 1,708 pts, ends below 48K	1
31519	lists 15% below the issue price	1
31540	Sensex up 642 pts, Nifty tad below 14750	1
31617	Nifty ends below 15,200	1
32122	Sensex dives 2%, Nifty ends below 11,400	1
32559		

In [11]:
#df.iloc[301, df.columns.get_loc('Sentiment')] = 1
df.at[5855, 'Sentiment'] = 0

In [10]:
df[df['Text'].str.contains('dividend') & (df['Sentiment']==1)][['Text','Sentiment']]

,Text,Sentiment
169,ITC gains 3% a day ahead of turning ex-dividend,1
1104,PTC India shareholders approve final dividend of Rs 5.80 per equity share,1
2798,Forbes & Co zooms 20% on special dividend cheer,1
3223,"Hindustan Zinc to pay Rs 21 per share dividend, outflow at Rs 8,873 cr",1
3832,ITC trades flat as stock turns ex-date for dividend of Rs 6.25 per share,1
3838,"Torrent Pharma rallies 9% on 1:1 bonus issue plan, dividend of Rs 23/share",1
5275,Hero Moto hits over 4-month high ahead of record date for interim dividend,1
5824,"This TVS group company declares ₹59 per share dividend, Q3 sales up 22%",1
5825,"Indraprastha Gas declares 150% dividend, records 67% jump in Q3 revenue",1
5855,"IT stock to pay interim dividend of ₹20 per share, record date next week",1
